# Mount Drive for dataset Load

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# Install Dependencies

In [2]:
# !pip install jiwer
# !pip install pandas
# !pip install tensorflow
# !pip install Numpy
# !pip install matplotlib



## Import Dependencies

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from IPython import display
from jiwer import wer

import pyaudio
import os
import wave
import librosa
# import numpy as np
from sys import byteorder
from array import array
from struct import pack
# import tensorflow as tf
import argparse
import speech_recognition as sr
import random


# Load The Dataset

In [4]:
data_path = r"C:/Users/purek/OneDrive/Desktop/Forkan last semester/Bangla_ASR_Done/Final_project/asr_data/data/asr-bengali-1000/asr_bengali_1000_wav"
wavs_path = data_path
metadata_path = r"C:/Users/purek/OneDrive/Desktop/Forkan last semester/Bangla_ASR_Done/Final_project/asr_data/data/asr-bengali-1000/data.csv"

# print(data_path)
# Read metadata file and parse it
metadata_df = pd.read_csv(metadata_path, sep=",")
metadata_df.columns = ["audio", "text"]
metadata_df = metadata_df[["audio", "text"]]
# print(metadata_df,": ")
metadata_df = metadata_df.sample(frac=1).reset_index(drop=True)
# print(metadata_df,"  2 : ")
metadata_df.head(11)



,audio,text
0,0a43bdd582.wav,২০১৩ সালের নভেম্বরের
1,0adf8db423.wav,মাথার বালিশ
2,0abd11911b.wav,জয়ললিতাকে তৃতীয় ফ্রন্টে চান
3,0a533992ec.wav,আগ্রহী হলেন
4,058a0299fd.wav,উদরস্থিত বায়ু
5,0a97949561.wav,শুধু আল্লাহকে মানলে
6,0a048b8ea0.wav,মানুষের জীবন যাত্রার উপর ছিল
7,05fdb2c6cd.wav,বর্গমূল কে বলা হয়
8,0a30e906fa.wav,বেসরকারি খাতের ১৯টি ব্যাংকও
9,05cf127b7e.wav,পুরুষের পরিচয়


In [5]:
# data_path = ""
# def data_gen(data_path):
#     data_path = r"C:\Users\purek\OneDrive\Desktop\Forkan last semester\Bangla_ASR_Done\gender-recognition-by-voice-master\asr_data\data\asr-bengali-1000\asr_bengali_1000_wav"
#     wavs_path = data_path
#     metadata_path = r"C:\Users\purek\OneDrive\Desktop\Forkan last semester\Bangla_ASR_Done\gender-recognition-by-voice-master\asr_data\data\asr-bengali-1000/data.csv"

#     # print(data_path)
#     # Read metadata file and parse it
#     metadata_df = pd.read_csv(metadata_path, sep=",")
#     metadata_df.columns = ["audio", "text"]
#     metadata_df = metadata_df[["audio", "text"]]
#     print(metadata_df,": ")
#     metadata_df = metadata_df.sample(frac=1).reset_index(drop=True)
#     print(metadata_df,"  2 : ")
#     metadata_df.head(5)

# Split the dataset into training and validation set

In [6]:
split = int(len(metadata_df) * .95)
df_train = metadata_df[:split]


split2 = int(len(metadata_df) * .05)
df_val = metadata_df[:split2]
# df_sex = metadata_df[:split2]

print(f"Size of the training set: {len(df_train)}")
print(f"Size of the validation set: {len(df_val)}")


Size of the training set: 948
Size of the validation set: 49


Test_Data

In [7]:
test_path = "H:/Final_project/asr_data/data/converted test"
wav_path = test_path
metadata_path_test = "H:/Final_project/asr_data/data/test.csv"

print(test_path)
# Read metadata file and parse it
test_data_df = pd.read_csv(metadata_path_test, sep="|")
test_data_df.columns = ["audio", "text"]
test_data_df = test_data_df[["audio", "text"]]
test_data_df = test_data_df.sample(frac=1).reset_index(drop=True)
test_data_df.head(5)

In [8]:
# split3 = int(len(test_data_df) * 1.0)
# df_test = test_data_df[:split3]


# print(f"Size of the testing set: {len(df_test)}")

## Preprocessing

We first prepare the vocabulary to be used.

In [9]:
chars_db_path = r'C:/Users/purek/OneDrive/Desktop/Forkan last semester/Bangla_ASR_Done/Final_project/asr_data/data/chars_75.csv'

unique_chars = pd.read_csv(chars_db_path, skip_blank_lines=False)['chars']
unique_chars = unique_chars.to_list()

# Mapping characters to integers
char_to_num = keras.layers.StringLookup(vocabulary=unique_chars, oov_token="")
# Mapping integers back to original characters
num_to_char = keras.layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True
)
print(
    f"The vocabulary is: {char_to_num.get_vocabulary()} "
    f"(size ={char_to_num.vocabulary_size()})"
)

The vocabulary is: ['', ' ', '।', 'ঁ', 'ং', 'ঃ', 'অ', 'আ', 'ই', 'ঈ', 'উ', 'ঋ', 'এ', 'ঐ', 'ও', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', '়', 'া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ে', 'ৈ', 'ো', 'ৌ', '্', 'ৎ', 'ড়', 'য়', '০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯'] (size =72)


# Spectogram and Label for preprocess

In [10]:

frame_length = 256
frame_step = 160
fft_length = 384


def encode_single_sample(wav_file, label):
    # Read wav file
    file = tf.io.read_file(data_path+"/"+wav_file)
    # Decode the wav file
    audio, _ = tf.audio.decode_wav(file)
    audio = tf.squeeze(audio, axis=-1)
    # Change type to float
    audio = tf.cast(audio, tf.float32)
    # Get the spectrogram
    spectrogram = tf.signal.stft(
        audio, frame_length=frame_length, frame_step=frame_step, fft_length=fft_length
    )
    # Magnitude value
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.math.pow(spectrogram, 0.5)
    # normalization the magnitude
    means = tf.math.reduce_mean(spectrogram, 1, keepdims=True)
    stddevs = tf.math.reduce_std(spectrogram, 1, keepdims=True)
    spectrogram = (spectrogram - means) / (stddevs + 1e-10)


    ##  Process the label

    # Convert label to Lower case
    label = tf.strings.lower(label)
    label = tf.strings.unicode_split(label, input_encoding="UTF-8")
    # Map the characters in label to numbers
    label = char_to_num(label)

    return spectrogram, label


## Creating `Dataset` objects

We create a `tf.data.Dataset` object that yields
the transformed elements, in the same order as they
appeared in the input.

In [11]:
batch_size = 5
# Define the trainig dataset
train_dataset = tf.data.Dataset.from_tensor_slices(
    (list(df_train["audio"]), list(df_train["text"]))
)
train_dataset = (
    train_dataset.map(encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE)
    .padded_batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)



# Define the validation dataset
validation_dataset = tf.data.Dataset.from_tensor_slices(
    (list(df_val["audio"]), list(df_val["text"]))
)


gender_dataset=validation_dataset

validation_dataset = (
    validation_dataset.map(encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE)
    .padded_batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)





## Model

CTC Loss function.

In [12]:

def CTCLoss(y_true, y_pred):
    # Compute the training-time loss value
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    loss = keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss


# ASR model

## Decoder & Callback 

In [13]:
# decode the output of the network
def decode_batch_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    # CTC_DECODE
    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0]
    # Iterate over the results and get back the text
    output_text = []
    for result in results:
        result = tf.strings.reduce_join(num_to_char(result)).numpy().decode("utf-8")
        output_text.append(result)
    return output_text

cnt=0
# A callback class to output a few transcriptions during training
class CallbackEval(keras.callbacks.Callback):
    """Displays a batch of outputs after every epoch."""

    def __init__(self, dataset):
        super().__init__()
        self.dataset = dataset

    def on_epoch_end(self, epoch: int, logs=None):
        # model_name="ASR_model_"+str(cnt)+".h5"
        # model.save(model_name)
        # cnt+=1
        predictions = []
        targets = []
        for batch in self.dataset:
            X, y = batch
            batch_predictions = model.predict(X)
            batch_predictions = decode_batch_predictions(batch_predictions)
            predictions.extend(batch_predictions)
            for label in y:
                label = (
                    tf.strings.reduce_join(num_to_char(label)).numpy().decode("utf-8")
                )
                targets.append(label)
        wer_score = wer(targets, predictions)
        print("-" * 100)
        print(f"Word Error Rate: {wer_score:.4f}")
        print("-" * 100)
        for i in np.random.randint(0, len(predictions), 2):
            print(f"Target    : {targets[i]}")
            print(f"Prediction: {predictions[i]}")
            print("-" * 100)


# Final Model Train

In [14]:

model=tf.keras.models.load_model("epoch100.h5",custom_objects={"CTCLoss":CTCLoss})
model.summary()

Model: "Bangla_ASR"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, None, 193)]       0         
_________________________________________________________________
expand_dim (Reshape)         (None, None, 193, 1)      0         
_________________________________________________________________
convolution_1 (Conv2D)       (None, None, 97, 3)       1353      
_________________________________________________________________
convolution_1_bn (BatchNorma (None, None, 97, 3)       12        
_________________________________________________________________
convolution_1_relu (ReLU)    (None, None, 97, 3)       0         
_________________________________________________________________
convolution_2 (Conv2D)       (None, None, 49, 3)       2079      
_________________________________________________________________
convolution_2_bn (BatchNorma (None, None, 49, 3)       1

#   Plot History of the model

In [15]:
# # Plot History
# def plot_history(history):

#     fig, axs = plt.subplots(2)
#     # accuracy sublpot
#     axs[0].plot(history.history["accuracy"], label="train accuracy")
#     axs[0].plot(history.history["val_accuracy"], label="test accuracy")
#     axs[0].set_ylabel("Accuracy")
#     axs[0].legend(loc="lower right")
#     axs[0].set_title("Accuracy eval")

#     # error sublpot
#     axs[1].plot(history.history["loss"], label="train error")
#     axs[1].plot(history.history["val_loss"], label="test error")
#     axs[1].set_ylabel("Error")
#     axs[1].set_xlabel("Epoch")
#     axs[1].legend(loc="upper right")
#     axs[1].set_title("Error eval")

#     plt.show()

In [16]:
# pd.DataFrame(history.history).plot(figsize=(8,5))
# plt.show()

In [17]:
# metrics = history.history
# plt.plot(history.epoch, metrics['loss'], metrics['val_loss'])
# plt.legend(['loss', 'val_loss'])
# plt.show()

In [18]:
# hist_df = pd.DataFrame(history.history) 

# # save to json:  
# # hist_json_file = 'history.json' 
# with open(hist_json_file, mode='w') as f:
#     hist_df.to_json(f)

In [19]:
def extract_feature(file_name, **kwargs):
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    X, sample_rate = librosa.core.load(file_name)
    if chroma or contrast:
        stft = np.abs(librosa.stft(X))
    result = np.array([])
    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result = np.hstack((result, mfccs))
    if chroma:
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result = np.hstack((result, chroma))
    if mel:
        mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        result = np.hstack((result, mel))
    if contrast:
        contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
        result = np.hstack((result, contrast))
    if tonnetz:
        tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
        result = np.hstack((result, tonnetz))
    return result

In [20]:
def sex(file_label):

    loc_csv=r"C:/Users/purek/OneDrive/Desktop/Forkan last semester/Bangla_ASR_Done/Final_project/asr_data/data/asr-bengali-1000/data.csv"
    data_loc=r"C:/Users/purek/OneDrive/Desktop/Forkan last semester/Bangla_ASR_Done/Final_project/asr_data/data/asr-bengali-1000/asr_bengali_1000_wav/"
    df = pd.read_csv(loc_csv, sep=',')
    cnt=0
    # data_loc=""
    for i in range(len(df)):
        a=df.iloc[i,0]
        b=df.iloc[i,1]
        # print(a,"\t",b)
        # print(df)
        # cnt+=1
        if b==file_label:
            # print(a,"\t",b)
            data_loc+=a
            # print(data_loc)
            break
    # return data_loc
    # print("Data Loc = ",data_loc)
    features = extract_feature(data_loc, mel=True).reshape(1, -1)
    modelsex = tf.keras.models.load_model("C:/Users/purek/OneDrive/Desktop/Forkan last semester/Bangla_ASR_Done/Final_project/results/model.h5")
    # predict the gender!
    male_prob = modelsex.predict(features)[0][0]
    # female_prob = 1 - male_prob
    # gender = "male" if male_prob > female_prob else "female"
    # print("Gender : ", gender)

    return male_prob

In [21]:
# gender_dataset = (
#     gender_dataset.map(encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE)
#     .padded_batch(1)
#     .prefetch(buffer_size=tf.data.AUTOTUNE)
# )

## Predictions

In [22]:
# Check results on more validation samples
predictions = []
targets = []
# gender = list()
for batch in validation_dataset:
    X, y = batch
    batch_predictions = model.predict(X)
    batch_predictions = decode_batch_predictions(batch_predictions)
    predictions.extend(batch_predictions)
    # print("binary val : ",y)
    for label in y:
        label = tf.strings.reduce_join(num_to_char(label)).numpy().decode("utf-8")
        targets.append(label)
        
        
wer_score = wer(targets, predictions)
print("-" * 100)
print(f"Word Error Rate: {wer_score:.4f}")
print("-" * 100)


for i in np.random.randint(0, len(predictions), 20):
    print(f"Target    : {targets[i]}")
    print(f"Prediction: {predictions[i]}")
    male = sex(targets[i])
    female=1-male
    # print("sex        : ",male)
    print("Sex       : Male: ",format(male*100,".2f"),"%     " ,"Female: ",format(female*100,".2f"),"%")
    # print(male)
    print("-" * 100)


----------------------------------------------------------------------------------------------------
Word Error Rate: 0.0748
----------------------------------------------------------------------------------------------------
Target    : একদমই দেখা যায়নি
Prediction: একদমই দেখা যায়নি
Sex       : Male:  13.86 %      Female:  86.14 %
----------------------------------------------------------------------------------------------------
Target    : তারা এ শিল্পকে নিয়ে
Prediction: তারা এ শিল্পকে নিয়ে
Sex       : Male:  94.06 %      Female:  5.94 %
----------------------------------------------------------------------------------------------------
Target    : হোয়াইট হাউজ
Prediction: হোয়াইট হাউজ
Sex       : Male:  7.92 %      Female:  92.08 %
----------------------------------------------------------------------------------------------------
Target    : সরকারের অনেক সময় লাগছে
Prediction: সরকার অনেক সময় লাগছে
Sex       : Male:  94.23 %      Female:  5.77 %
--------------------------------